In [ ]:
import pathlib

import pandas as pd
from IPython.display import display
import yaml

import dlsproc.bundle
import dlsproc.hier
import dlsproc.postprocess
import dlsproc.assemble

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data


## Outsiders

> Licitaciones publicadas en la Plataforma mediante mecanismos de agregación, excluyendo los contratos menores

A file to *cache* the data read

In [ ]:
outsiders_output_file = dlsproc.io.File('outsiders')

The directory for *outsiders* data

In [ ]:
# outsiders_directory = data_directory / 'agregados'
outsiders_directory = pathlib.Path.cwd().parent / 'samples' / 'yearly'
assert outsiders_directory.exists()
print(outsiders_directory)

/home/manu/dlsproc/samples/yearly


In [ ]:
zip_files = [
    'PlataformasAgregadasSinMenores_2018.zip',
    'PlataformasAgregadasSinMenores_2019.zip',
    'PlataformasAgregadasSinMenores_2020.zip',
    'PlataformasAgregadasSinMenores_2021.zip'
]

In [ ]:
res_df = None
res_deleted_series = None

In [ ]:
for filename in zip_files:
    
    print(filename)
    
    # above file names are relative to a directory
    f = outsiders_directory / filename
    
    assert f.exists()
    
    # data is read from the above *zip* file, and concatenated into a single `pd.DataFrame`...
    yearly_df = dlsproc.bundle.read_zip(f, concatenate=True)
    
    # ...and re-structured with multiindexed columns
    yearly_df = dlsproc.hier.flat_df_to_multiindexed_df(yearly_df)
    
    # the same contract might show up more than once due to updates...but only the last one is kept
    last_update_only_df = dlsproc.postprocess.keep_updates_only(yearly_df)
    
    # the same zip file also contains information (at the beginning) about deleted entries
    deleted_series = dlsproc.bundle.read_deleted_zip(f)
    
    # if this is NOT the first iteration...
    if res_df is not None:
    
        # ...the new data is stacked
        res_df = dlsproc.assemble.stack(res_df, last_update_only_df)
        res_deleted_series = pd.concat((res_deleted_series, deleted_series), axis=0)
        
    # ...if this is the first iteration
    else:
        
        # ...the new data is set as the accumulated result
        res_df = last_update_only_df
        res_deleted_series = deleted_series

PlataformasAgregadasSinMenores_2018.zip
PlataformasAgregadasSinMenores_2019.zip
PlataformasAgregadasSinMenores_2020.zip
PlataformasAgregadasSinMenores_2021.zip


In [ ]:
res_df.shape

(5062, 40)

In [ ]:
deduplicated_deleted_series = res_deleted_series.sort_values().groupby(res_deleted_series.index).tail(1)
deduplicated_deleted_series

file name                                              id                                                                                 
PlataformasAgregadasSinMenores_20180217_180137_1.atom  https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1974509   2017-12-31 13:10:39.114000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1970608   2017-12-31 13:10:39.180000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1973692   2017-12-31 14:10:39.150000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1975721   2017-12-31 14:10:39.242000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1974172   2017-12-31 14:10:39.3680

In [ ]:
stateful_df = dlsproc.assemble.merge_deleted(res_df, deduplicated_deleted_series)
stateful_df

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20180217_180137_... https://contrataciondelestado.es/sindicacion/Pl...   453   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   452   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   451   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   450   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   449   
..                                                                                                      ...   
PlataformasAgregadasSinMenores_20210106_030028.... https://contrataciondelestado.es/sindicacion/Pl...     4   
                                                   https://contrataciondelestado.es/sindicacion/Pl...     3   
                                                   https://contrataciondelestado.es/sindicacion/Pl...     2   
                                                   https://contrataciondelestado.es/sindicacion/Pl...     1   
                                                   https://contrataciondelestado.es/sindicacion/Pl...     0   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20180217_180137_... https://contrataciondelestado.es/sindicacion/Pl...  Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Expediente: 1282/17, Entidad: Diputación Provi...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Expediente: 1281/17, Entidad: Diputación Provi...   
         